# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [3]:
#Needed if running on local
!pip install pandas
!pip install psycopg2

In [4]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

/home/mb/udacity/de/postgres/postgres/postgres/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [5]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [7]:
song_files = get_files("data/song_data");
#print(song_files)

In [8]:
filepath = song_files[0];
#print(filepath)

In [9]:
df = pd.read_json(filepath, lines=True)
#print(df)

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
#print(song_data)

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [11]:
%run -i 'create_tables.py'

OperationalError: database "sparkifydb" is being accessed by other users
DETAIL:  There is 1 other session using the database.


In [12]:
#cur.execute(song_table_insert, ("S", "didnt mean to", "hi", 0, 208))
try: 
    cur.execute(song_table_insert, song_data)
            #("S", "didnt mean to", "hi", 0, 208)
    conn.commit()
    print("Success")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Success


Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [13]:
#TODO: lat / long are NaN; better to put "NULL"
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()

#print(artist_data)

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [14]:
try: 
    cur.execute(artist_table_insert, artist_data)
    conn.commit()
    print("Success")
except psycopg2.Error as e: 
    print("Error: Inserting Rows")
    print (e)

Success


Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [15]:
log_files = get_files("data/log_data");

In [16]:
filepath = log_files[0];
print(filepath)

/home/mb/udacity/de/postgres/postgres/song_project/data/log_data/2018/11/2018-11-20-events.json


In [17]:
df = pd.read_json(filepath, lines=True)
#print(df.head())

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [18]:
df['ts'].head()

0    1542672042796
1    1542672288796
2    1542672531796
3    1542672801796
4    1542672935796
Name: ts, dtype: int64

In [19]:
#Filter records by page = "NextSong" action
#408 rows, once filtered out, XX rows
#TODO check filter is working correctly

df = df[df['page'] =='NextSong']
df.dtypes
#print(df.head())

artist            object
auth              object
firstName         object
gender            object
itemInSession      int64
lastName          object
length           float64
level             object
location          object
method            object
page              object
registration     float64
sessionId          int64
song              object
status             int64
ts                 int64
userAgent         object
userId            object
dtype: object

In [20]:
# Convert the ts timestamp column to datetime
# The current timestamp is in milliseconds

t = pd.to_datetime(df['ts'], unit ='ms')
t.head()

0   2018-11-20 00:00:42.796
1   2018-11-20 00:04:48.796
2   2018-11-20 00:08:51.796
3   2018-11-20 00:13:21.796
4   2018-11-20 00:15:35.796
Name: ts, dtype: datetime64[ns]

In [21]:
#Specify labels for these columns and set to column_labels
#Create a dataframe, time_df, containing the time data for this 
#file by combining column_labels and time_data into a dictionary 
#and converting this into a dataframe

#TODO: come up with better solution
#time_data = (timestamp, hour, month, day, weekofyear, year, weekday)

time_data = (t, t.dt.hour, t.dt.day, t.dt.week, t.dt.month, t.dt.year, t.dt.weekday)
column_labels = ('start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday')

#Create dataframe
#time_df = pd.DataFrame(column_labels, time_data)


#https://stackoverflow.com/questions/39941321/create-dataframe-from-multiple-series
time_df = pd.DataFrame(dict(zip(column_labels, time_data)));

time_df.head()


,start_time,hour,day,week,month,year,weekday
0,2018-11-20 00:00:42.796,0,20,47,11,2018,1
1,2018-11-20 00:04:48.796,0,20,47,11,2018,1
2,2018-11-20 00:08:51.796,0,20,47,11,2018,1
3,2018-11-20 00:13:21.796,0,20,47,11,2018,1
4,2018-11-20 00:15:35.796,0,20,47,11,2018,1


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [22]:
time_df.dtypes

start_time    datetime64[ns]
hour                   int64
day                    int64
week                   int64
month                  int64
year                   int64
weekday                int64
dtype: object

In [33]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

for i, row in time_df.iterrows():
    #print(time_df)
    if i == 0:
        print(i)
        print(row)
        print(list(row))
        cur.execute(time_table_insert, list(row))
        conn.commit()

0
start_time    2018-11-20 00:00:42.796000
hour                                   0
day                                   20
week                                  47
month                                 11
year                                2018
weekday                                1
Name: 0, dtype: object
[Timestamp('2018-11-20 00:00:42.796000'), 0, 20, 47, 11, 2018, 1]


Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = pd.DataFrame({'user_id': df['userId'],'first_name': df['firstName'], \
                       'last_name': df['lastName'], 'gender': df['gender'], \
                       'level': df['level']})

user_df.head()

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added records to this table.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    #results = cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.execute(song_select, (row.song, row.artist, row.length))
    songid, artistid = results if results else None, None

    # insert songplay record
    songplay_data = (index, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
          
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.